# Inverted File Index

In [ ]:
using HDF5
using BenchmarkTools
using Distances
using LoopVectorization
using SIMD
using Clustering
using ProgressMeter
using StaticArrays
using DataFrames
using Plots
using NPZ

## Loading SIFT1M dataset

In [ ]:
#path = joinpath(homedir(), "Datasets", "SIFT1M",  "sift-128-euclidean.hdf5")
#path = joinpath(homedir(), "TFM", "ann-benchmarks",  "sift-128-euclidean.hdf5")

# david
path = joinpath(homedir(), "Datasets", "SIFT1M", "sift-128-euclidean.hdf5")

In [ ]:
f = h5open(path, "r")

X_tr_vecs = read(f["train"])
X_te_vecs = read(f["test"]);
true_neighbors = read(f["neighbors"])
true_distances = read(f["distances"])

true_neighbors .= true_neighbors .+ 1;

@show size(X_tr_vecs)
@show size(X_te_vecs)
@show size(true_neighbors)
@show size(true_distances)

n_features, n_examples = size(X_tr_vecs)

## Generating shared PQ-codes

In [ ]:
function encode_shared(dist, vector::Array{T}, shared_prototypes::Array{T}) where T
    n_clusters = length(shared_prototypes)
    n_features = size(vector)[1]
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(shared_prototypes[k], x)
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
           #println(k, ' ', j, ' ', best_coordinate, ' ',min_distance )
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

In [ ]:
P_shared = vec(Float32.(npzread("1dkmeans_prototypes/1dkmeans_shared_prototypes_SIFT1M.npy")))

In [ ]:
PQcodes_shared = Array{Int8}(undef, n_features, n_examples);

for j in 1:n_examples
    PQcodes_shared[:,j] = encode_shared(euclidean, X_tr_vecs[:,j], P_shared)  
end

## IVF indexer

In [ ]:
function ivf_indexer(X, PQcodes, n_cells, clustering_function)
    n_features, n_examples = size(X)

    result = clustering_function(X, n_cells)
    centroids = result.centers
    cell_ids = result.assignments
    
    cells = Vector{Matrix{Int8}}(undef, n_cells)
    pq_ids = Vector{Vector{Int32}}(undef, n_cells)
    @inbounds @fastmath for i in 1:n_cells
        pq_idx = findall(x -> x == i,cell_ids) 
        cells[i] = PQcodes[:,pq_idx]
        pq_ids[i] =  pq_idx 
    end
    return centroids, cells, pq_ids
end

In [ ]:
n_cells = 100
clustering_function = kmeans
@time centroids, cells, indexes  = ivf_indexer(X_tr_vecs, PQcodes_shared, n_cells, clustering_function)

## IVF search

We define some standard

In [ ]:

function Euclidean0(x, query)
    @assert length(x) == length(query)
    res = zero(eltype(x))
    @inbounds  @simd for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return sqrt(res)
end

In [ ]:
function recall(predicted, relevant, eval_at)
    """
    fraction of the relevant documents that are successfully retrieved
    """
    if eval_at == 0
        return 0.0
    end
    
    predicted_at_k = predicted[1:eval_at]
    n_predicted_and_relevant =  length(intersect( Set(predicted_at_k), Set(relevant))) 
    return n_predicted_and_relevant/ length(relevant)
end

@inline function Euclidean0(x, query)
    @assert length(x) == length(query)
    res = zero(eltype(x))
    @inbounds  @simd for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return sqrt(res)
end


function adc_dist_shared( x_code,  adc_table::Matrix)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(x_code)
        res+= adc_table[x_code[j], j]
    end
    return res
end


function adc_dist_mat( x_code,  adc_table::Matrix, k)
    res = zero(eltype(adc_table))
    @inbounds @simd for j in eachindex(x_code)
        res+= adc_table[x_code[j, k], j]
    end
    return res
end

function compute_ADC_shared(query, prototypes, dist)
    """
    Computes the distance between each query[k] and prototype[k]
    
    Arguments:
    
    - y (Array{T}): vector of n_features components.
    - prototypes (Array{T}): vector of n_cluster components.
    - dist (function): distance to be used to compare prototypes and query.
    
    """
    #@assert ndims(prototypes) ==1
    
    n_features = length(query)
    n_clusters = length(prototypes)
    ADC_table = Array{Float32}(undef, n_clusters, n_features)
    
    for j in 1:n_features       # 128
        for p in 1:n_clusters   # 32
            ADC_table[p,j] = dist(query[j], prototypes[p] )
        end
    end
    @assert ndims(ADC_table)==2
    return  ADC_table
end

@inline function abs_dist(y::Array{T}, X::Array{T}, j) where T
    # Here I use a bigger Int type than 8 due to avoid
    # res beeing overflowed
    res = Int16(0)
    @inbounds @fastmath  for k in eachindex(y)
        res += abs(X[k, j] - y[k])
    end
    return res
end


@inline function euclidean_mat2(y, X, j) where T
    # Here I use a bigger Int type than 8 due to avoid
    # res beeing overflowed
    res = zero(eltype(y))
    @inbounds @fastmath  for k in eachindex(y)
        partial = X[k, j] - y[k]
        res += partial * partial
    end
    return res
end

In [ ]:
@inline function abs_dist(y::Array{T}, X::Array{T}, j) where T
    # Here I use a bigger Int type than 8 due to avoid
    # res beeing overflowed
    res = Int16(0)
    @inbounds @fastmath  for k in eachindex(y)
        res += abs(X[k, j] - y[k])
    end
    return res
end

function select_cells(n_cells, nprobe, centroids, query, dist)
    distances = Array{Float32}(undef, n_cells)
    for i in 1:n_cells
        distances[i] = dist(query, centroids[:,i]) 
    end
    closest_prototypes = partialsortperm(distances, 1:nprobe)
    best_distances = distances[closest_prototypes]

    return best_distances, closest_prototypes
end


function get_top_distances(query, X, prototypes, cells, indexes, fine_quant_funct, P, top_k, extra_factor, refinement, j)

    real_index = prototypes[j]
    pq_codes = cells[real_index]
    original_ids = indexes[real_index]
    pq_distances = fine_quant_funct(query, pq_codes, P)
    n_to_sort = minimum([top_k*extra_factor, length(pq_distances)])
    top_k_pq = partialsortperm(pq_distances, 1:n_to_sort) 

    #Exact search if refinement is chosen
    if refinement == true
        best_ids = Vector{Int32}(undef, n_to_sort)
        for i in 1:n_to_sort
            id = top_k_pq[i]
            best_ids[i] = original_ids[id]
        end
        pq_distances = linear_scann_exact(query, X[:,best_ids]); #X_tr_vecs in memory, SOLVE THIS ->memmap, HDF5
        top_k_pq = partialsortperm(pq_distances, 1:top_k)
        original_ids = best_ids
    end

    #retrieve original ids and distances of closest vectors
    ids = Vector{Int32}(undef, top_k)
    distances = Vector{Float32}(undef, top_k)
    for i in 1:top_k
        id = top_k_pq[i]
        ids[i] = original_ids[id]
        distances[i] = pq_distances[id]
    end
    return distances, ids
end


function top_distances(query, X, closest_prototypes, cells, indexes, fine_quant_funct, P, top_k, extra_factor, refinement, nprobe)
    result_distances = Vector{Vector{Float32}}(undef, nprobe)
    result_ids = Vector{Vector{Int32}}(undef, nprobe)
    for i in 1:nprobe
        distances, ids = get_top_distances(query, X, closest_prototypes, cells, indexes, fine_quant_funct, P, top_k, extra_factor, refinement, i)
        result_distances[i] = distances
        result_ids[i] = ids
    end
    return result_distances, result_ids
end


function linear_scann_shared_l1(query, PQcodes_shared, P_shared)

    n_features, n_examples = size(PQcodes_shared)
    #distances = Array{Float32}(undef, n_examples)
    distances = Array{UInt16}(undef, n_examples) # David Update, we do not need conversion now

    query_code = encode_shared(euclidean, query, P_shared)
    
    @inbounds @fastmath for j in 1:n_examples
        distances[j] = abs_dist(query_code, PQcodes_shared, j)    
    end
    return distances
end

function linear_scann_shared_ADC(query, PQcodes_shared, P_shared)
    
    n_features, n_examples = size(PQcodes_shared)
    distances = Array{eltype(query)}(undef, n_examples)
    adc_table_shared = compute_ADC_shared(query, P_shared, euclidean)
    
    @inbounds @fastmath for j in 1:n_examples
        #distances[j] = adc_dist_shared(view(PQcodes_shared,:,j) ,  adc_table_shared)  
        distances[j] = adc_dist_mat(PQcodes_shared,  adc_table_shared, j )  
    end
    return distances
end

function linear_scann_exact(query, X)

    n_features, n_examples = size(X)
    distances = Array{Float32}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = euclidean_mat2(query, X, j)    
    end
    return distances
end


function priority_queue_sort(distances, ids, top_k)
    top_k_distances = distances[1]
    top_k_vectors = ids[1]
    for k in 2:length(distances)
        for i in 1:length(distances[k])
            if distances[k][i] < top_k_distances[top_k]
                j = top_k
                while distances[k][i] < top_k_distances[j-1]
                    j = j-1
                    if j == 1
                        break
                    end
                end
                top_k_distances_fixed = copy(top_k_distances)
                top_k_vectors_fixed = copy(top_k_vectors)
                for element in j+1:top_k
                    top_k_distances[element] = top_k_distances_fixed[element-1]
                    top_k_vectors[element] = top_k_vectors_fixed[element-1]
                end
                top_k_distances[j] = distances[k][i]
                top_k_vectors[j] = ids[k][i]
            else
                break
            end
        end
    end
    return top_k_distances, top_k_vectors
end

In [ ]:
function ivf_search(query, X, centroids, cells, indexes, dist, nprobe, PQcodes, P, fine_quant_funct, top_k, extra_factor, refinement)

    if refinement == false
        @assert extra_factor==1
    end

    #Get nprobe closest centroids 
    n_cells = length(cells)
    best_distances, closest_prototypes = select_cells(n_cells, nprobe, centroids, query, dist)
    
    #Get top_k distances in each selected cell
    result_distances, result_ids = top_distances(query, X, closest_prototypes, cells, indexes, fine_quant_funct, P, top_k, extra_factor, refinement, nprobe)
     
    #Get top_k closest distances
    top_k_distances, top_k_vectors = priority_queue_sort(result_distances, result_ids, top_k)

    return top_k_distances, top_k_vectors
end

In [ ]:
query_id = 5
query = X_te_vecs[:,query_id];
top_k = 100
query_true_neighbors = true_neighbors[:,query_id][1:top_k]
X = X_tr_vecs
nprobe = 5
dist = Euclidean0
refinement = true
extra_factor = 2
fine_quant_funct = linear_scann_shared_l1
#fine_quant_funct = linear_scann_shared_ADC

top_k_distances, top_k_vectors = ivf_search(query, X, centroids, cells, indexes, dist, nprobe, PQcodes_shared, P_shared, fine_quant_funct, top_k, extra_factor, refinement)
@show recall(top_k_vectors, query_true_neighbors, top_k)

In [ ]:
@benchmark ivf_search($query, $X, $centroids, $cells, $indexes, $Euclidean0, $nprobe, $PQcodes_shared, $P_shared, $fine_quant_funct, $top_k, $extra_factor, $refinement)

In [ ]:
query_id = 5
query = X_te_vecs[:,query_id];
top_k = 100
query_true_neighbors = true_neighbors[:,query_id][1:top_k]
X = X_tr_vecs
nprobe = 7
dist = Euclidean0
refinement = true
extra_factor = 2
#fine_quant_funct = linear_scann_shared_l1
fine_quant_funct = linear_scann_shared_ADC

top_k_distances, top_k_vectors = ivf_search(query, X, centroids, cells, indexes, dist, nprobe, PQcodes_shared, P_shared, fine_quant_funct, top_k, extra_factor, refinement)
@show recall(top_k_vectors, query_true_neighbors, top_k)

In [ ]:
@benchmark ivf_search($query, $X, $centroids, $cells, $indexes, $Euclidean0, $nprobe, $PQcodes_shared, $P_shared, $fine_quant_funct, $top_k, $extra_factor, $refinement)

In [ ]:
query_id = 1
query = X_te_vecs[:,query_id];
query_true_neighbors = true_neighbors[:,query_id]
X = X_tr_vecs
top_k = 100
nprobe = 20
dist = Euclidean0
refinement = true
extra_factor = 10
fine_quant_funct = linear_scann_shared_l1

function compute_mean_recall_and_time_IVF(Q, X, centroids, cells, indexes, dist, nprobe, PQcodes_shared, P_shared, fine_quant_funct, top_k, extra_factor, refinement)
    n_features, n_examples = size(Q)
    rec = 0.
    t1 = 0.
    t2 = 0.
    t3 = 0.
    for query_id in 1:n_examples
        query = Q[:, query_id]
        query_true_neighbors = true_neighbors[:,query_id]
        #Get nprobe closest centroids 
        n_cells = length(cells)
        t = @elapsed best_distances, closest_prototypes = select_cells(n_cells, nprobe, centroids, query, dist)
        t1 += t;
        
        #Get top_k distances in each selected cell
        t = @elapsed result_distances, result_ids = top_distances(query, X, closest_prototypes, cells, indexes, fine_quant_funct, P_shared, top_k, extra_factor, refinement, nprobe)
        t2 += t;
        #Get top_k closest distances
        t = @elapsed top_k_distances, top_k_vectors = priority_queue_sort(result_distances, result_ids, top_k)
        t3 += t;
        r = recall(top_k_vectors, query_true_neighbors, top_k);
        rec += r;
        end
    return rec/n_examples, t1/n_examples, t2/n_examples, t3/n_examples
    end

avg_recall, avg_time1, avg_time2, avg_time3 = compute_mean_recall_and_time_IVF(X_te_vecs[:, 1:100], X, centroids, cells, indexes, dist, nprobe, PQcodes_shared, P_shared, fine_quant_funct, top_k, extra_factor, refinement) 

## Plots for n_cells = 100

In [ ]:
recall_array_100 = []
times_100_1 = []
times_100_2 = []
times_100_3 = []
for nprobe in 1:20
    avg_recall, avg_time1, avg_time2, avg_time3 = compute_mean_recall_and_time_IVF(X_te_vecs[:, 1:100], X, centroids, cells, indexes, dist, nprobe, PQcodes_shared, P_shared, fine_quant_funct, top_k, extra_factor, refinement) 
    push!(recall_array_100, avg_recall)
    push!(times_100_1, avg_time1)
    push!(times_100_2, avg_time2)
    push!(times_100_3, avg_time3)
end

In [ ]:
x = 1:20
y = recall_array_100
plot(x, y, title = "IVF search recall | 100 cells", legend = false)
xlabel!("nprobe")
ylabel!("Recall")

In [ ]:
y = [times_100_1*1000, times_100_2*1000, times_100_3*1000]
plot(x, y, title = "Times IVF search | 100 cells", label = ["Cell Selection" "Distances calculation" "Top k Distances"], legend=:topleft)
xlabel!("nprobe")
ylabel!("Time (ms)")

Let us look into the times for each function involved in the distance calculation:

In [ ]:
nprobe = 10
i = 10
@benchmark best_distances, closest_prototypes = select_cells(n_cells, nprobe, centroids, query, dist)

In [ ]:
nprobe = 10
i = 10
best_distances, closest_prototypes = select_cells(n_cells, nprobe, centroids, query, dist)
@benchmark get_top_distances(query, X, closest_prototypes, cells, indexes, fine_quant_funct, P_shared, top_k, extra_factor, refinement, i)

In [ ]:
aux = get_top_distances(query, X, closest_prototypes, cells, indexes, fine_quant_funct, P_shared, top_k, extra_factor, refinement, i)

In [ ]:
real_index = closest_prototypes[i]
pq_codes = cells[real_index]
original_ids = indexes[real_index]
@benchmark pq_distances = fine_quant_funct(query, pq_codes, P_shared)  

In [ ]:
## TODO: could we avoid this sort if we got pq_distances already sorted ?
pq_distances = fine_quant_funct(query, pq_codes, P_shared)  
n_to_sort = minimum([top_k*extra_factor, length(pq_distances)])
@benchmark top_k_pq = partialsortperm(pq_distances, 1:n_to_sort) 

In [ ]:
top_k_pq = partialsortperm(pq_distances, 1:n_to_sort) 
best_ids = Vector{Int32}(undef, n_to_sort)
for i in 1:n_to_sort
    id = top_k_pq[i]
    best_ids[i] = original_ids[id]
end

@benchmark linear_scann_exact(query, X[:,best_ids])

In [ ]:
@benchmark linear_scann_exact(query, view(X,:,best_ids))

In [ ]:
pq_distances = linear_scann_exact(query, X[:,best_ids])
@benchmark top_k_pq = partialsortperm(pq_distances, 1:top_k)

## Plots for n_cells = 256

In [ ]:
recall_array_256 = []
times_256_1 = []
times_256_2 = []
times_256_3 = []
for nprobe in 1:20
    avg_recall, avg_time1, avg_time2, avg_time3 = compute_mean_recall_and_time_IVF(X_te_vecs[:, 1:100], X, centroids, cells, indexes, dist, nprobe, PQcodes_shared, P_shared, fine_quant_funct, top_k, extra_factor, refinement) 
    push!(recall_array_256, avg_recall)
    push!(times_256_1, avg_time1)
    push!(times_256_2, avg_time2)
    push!(times_256_3, avg_time3)
end

In [ ]:
x = 1:20
y = recall_array_256
plot(x, y, title = "IVF search recall | 256 cells", legend = false)
xlabel!("nprobe")
ylabel!("Recall")

In [ ]:
y = [times_256_1*1000, times_256_2*1000, times_256_3*1000]
plot(x, y, title = "Times IVF search | 256 cells", label = ["Cell Selection" "Distances calculation" "Top k Distances"], legend=:topleft)
xlabel!("nprobe")
ylabel!("Time (ms)")

## Plots for n_cells = 1000

In [ ]:
recall_array_1000 = []
times_1000_1 = []
times_1000_2 = []
times_1000_3 = []
for nprobe in 1:20
    avg_recall, avg_time1, avg_time2, avg_time3 = compute_mean_recall_and_time_IVF(X_te_vecs[:, 1:100], X, centroids, cells, indexes, dist, nprobe, PQcodes_shared, P_shared, fine_quant_funct, top_k, extra_factor, refinement) 
    push!(recall_array_1000, avg_recall)
    push!(times_1000_1, avg_time1)
    push!(times_1000_2, avg_time2)
    push!(times_1000_3, avg_time3)
end

In [ ]:
x = 1:20
y = recall_array_1000
plot(x, y, title = "IVF search recall | 1000 cells", legend = false)
xlabel!("nprobe")
ylabel!("Recall")

In [ ]:
y = [times_1000_1*1000, times_1000_2*1000, times_1000_3*1000]
plot(x, y, title = "Times IVF search | 1000 cells", label = ["Cell Selection" "Distances calculation" "Top k Distances"], legend=:topleft)
xlabel!("nprobe")
ylabel!("Time (ms)")